In [33]:
import pandas as pd 
from dash import Dash, html, Input, Output,callback, dcc
import dash 
casos = pd.read_csv('caso_full.csv', sep = ',')


In [10]:
casos.head()

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
0,Rio Branco,1200401.0,2020-03-17,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-17,0.0,0,1,city,AC,3,0
1,NaN,12.0,2020-03-17,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-17,0.0,0,1,state,AC,3,0
2,Rio Branco,1200401.0,2020-03-18,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-18,0.0,0,2,city,AC,0,0
3,NaN,12.0,2020-03-18,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-18,0.0,0,2,state,AC,0,0
4,Rio Branco,1200401.0,2020-03-19,202012,413418.0,407319.0,False,False,4,0.96754,2020-03-19,0.0,0,3,city,AC,1,0


In [13]:
casos['data'] = pd.to_datetime(casos['date'])
casos['ano'] = casos['data'].dt.year
casos['mes'] = casos['data'].dt.month

In [14]:
casos = casos[['city', 'epidemiological_week', 'state', 'new_confirmed', 'new_deaths', 'ano','mes']]

In [19]:
casos.head()

,city,epidemiological_week,state,new_confirmed,new_deaths,ano,mes
0,Rio Branco,202012,AC,3,0,2020,3
1,NaN,202012,AC,3,0,2020,3
2,Rio Branco,202012,AC,0,0,2020,3
3,NaN,202012,AC,0,0,2020,3
4,Rio Branco,202012,AC,1,0,2020,3


In [20]:
casos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3853648 entries, 0 to 3853647
Data columns (total 7 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   city                  object
 1   epidemiological_week  int64 
 2   state                 object
 3   new_confirmed         int64 
 4   new_deaths            int64 
 5   ano                   int32 
 6   mes                   int32 
dtypes: int32(2), int64(3), object(2)
memory usage: 176.4+ MB


In [32]:
total_ano = casos[['state','new_deaths','ano','new_confirmed']].groupby(['state','ano']).sum().reset_index()
total_estado = casos[['state','new_deaths','new_confirmed']].groupby('state').sum().reset_index()
total_mes = casos[['state','mes','new_deaths','new_confirmed']].groupby(['state','mes']).sum().reset_index()


In [61]:
import plotly.express as px
app = Dash(__name__)

app.layout = html.Div([

    html.Div(children=[
    # Griando botão de filtro para os gráficos
    dcc.Dropdown(
        id ='filtro_ano',
        options = [{'label': ano, 'value': ano} for ano in total_ano['ano'].unique()],
        value = 2020 ),


        dcc.Dropdown(
            id ='filtro_estados',
            options = [{'label': estado, 'value': estado} for estado in total_estado['state'].unique()],
            value = 'SP'), 

    html.Div(children=[
        dcc.Graph(id = 'mapa_calor'),
        dcc.Graph(id = 'mapa_linhas')

    ])
    
    ]),
    
])

@callback(
    Output('mapa_calor','figure'),
     
    #  Output('mapa_linhas','fiugre')]
     Input('filtro_estados','value'))
    #   Input('mapa_linhas','value')])

def atualiza_charts(estado):
    estado_df = total_estado[total_estado['state'] == estado]
    fig_mapa =  px.choropleth(estado_df,
                             geojson='https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson',
                             locations='state',
                             featureidkey='properties.sigla',
                             color='state',
                             hover_name='state',
                             hover_data=['new_confirmed','new_deaths'],
                             color_continuous_scale='Blues',
                             title="Mapa Coroplético dos Casos por Estado no Brasil")
    
    return fig_mapa 

app.run_server(debug=True)